# Quantum Molecular Dynamics 

“*In fact, the mere act of opening the box will determine the state of the
cat, although in this case there were three determinate states the cat
could be in: these being* **Alive**, **Dead**, *and* **Bloody Furious**.”
― Terry Pratchett, "Lords and Ladies" 

## High-Level Problem Description

TODO

## Mathematical Foundations

### Input Matrix ###

 The input matrix $H$, which represents the state of the quantum system, is built in the following way:

Let $A$, $B$ be two segments with $n$ points. We define $H$ as:

\begin{equation}
H = \begin{pmatrix}
        0^{n \times n} & D(A,B) \\
        D(A,B)^{T} & 0^{n \times n} \\
     \end{pmatrix}
\end{equation}

Such that each $D_{AB}$ cell contains the **squared euclidean distance** between each component of $A$ and $B$, namely,

\begin{equation}
d(a_i,b_i) = \sqrt{(b_x - a_x)^2 + (b_y - a_y)^2 + (b_z - a_z)^2}.
\label{eq:distanceEq}
\end{equation}

**Th**: Input matrix $H$ is **Hermitian**.

**Proof**: Let $H \in \mathbb{C}^{n} \times \mathbb{C}^{n}$, with $n \in \mathbb{N}$. $H$ is **Hermitian** $\iff H = H^{\dagger}$.

$H^{\dagger}$ is the **adjunct** of $H$, i.e., the **transpose complex conjugate** of $H$, meaning that $H_{ij} = \overline{H_{ji}}$. 

For $0$ cells, complex conjugate is 0. For cells of $D_{AB}$, we know that $d$ is defined as an application $\mathbb{V} \times \mathbb{V} \mapsto \mathbb{R}$. Since for each $x \in \mathbb{R}, \overline{x} = x$, we have that $H = \overline{H}$.

Now, what's left to prove is that $H = H^{T}$. To prove it, we calculate the transpose of $H$ as follows:

$H^{T} = \begin{pmatrix}
           0 & \dots & 0 & D(A,B)_{0,0} & \dots & D(A,B)_{0,n} \\
           \vdots & \ddots & \vdots & \vdots & \ddots & \vdots \\
           0 & \dots & 0 & D(A,B)_{n,0} & \dots & D(A,B)_{n,n}  \\
           D(A,B)_{0,0} & \dots & D(A,B)_{n,0} & 0 & \dots & 0\\
           \vdots & \ddots & \vdots & \vdots & \ddots & \vdots \\
           D(A,B)_{0,n} & \dots & D(A,B)_{n,n} & 0 & \dots & 0\\
        \end{pmatrix}^{T} = \begin{pmatrix}
           0 & \dots & 0 & D(A,B)_{0,0} & \dots & D(A,B)_{0,n} \\
           \vdots & \ddots & \vdots & \vdots & \ddots & \vdots \\
           0 & \dots & 0 & D(A,B)_{n,0} & \dots & D(A,B)_{n,n}  \\
           D(A,B)_{0,0} & \dots & D(A,B)_{n,0} & 0 & \dots & 0\\
           \vdots & \ddots & \vdots & \vdots & \ddots & \vdots \\
           D(A,B)_{0,n} & \dots & D(A,B)_{n,n} & 0 & \dots & 0\\
        \end{pmatrix}$

Since $H_{i,j}^{T} = H_{j,i}.~\mathbf{\square}$

The main advantage of $H$ being Hermitian is that all the eigenvalues are real, therefore are suitable representations for the Hamiltonians of quantum systems.

### Eigenvalues Calculation 

The goal of this work is to calculate distance of two alpha carbon atoms in the same segment or among two different segments. To this end, we need to calculate the lowest eigenvalue of $H$. 

Calculation of eigenvalues in quantum machine is defined as follows: given a Hermitian matrix 𝐻
and a minimum eigenvalue $\lambda_{min}$, associated to a quantum eigenstate $|\psi_{min}\rangle$, we want to find a $\lambda_{\theta}$ such that:

\begin{equation}
\lambda_{min} \leq \lambda_{\theta} \equiv \langle \psi(\theta)|H|\psi(\theta)\rangle
\end{equation}

Where $\psi(\theta)$ is the eigenstate associated to $\lambda_{\theta}$ and $\theta$ is the parameter of the quantum Ansatz, which is used to minimize $\lambda_{\theta}$.

In order to find $\lambda_{\theta}$, we define the problem as the **minimization** of the cost function $C(\theta)$, defined as:

\begin{equation}
C(\theta) = \langle \psi(\theta) | H | \psi(\theta) \rangle
\end{equation}

While the minimization of $C$ can be solved by a standard optimizer, i.e., COBYLA, SPSS, we need to define a **parametrized quantum circuit** for the state $\psi(\theta)$. 

### Parametrized Circuit Definition

TODO
* Check U gate
* Entanglement?